In [1]:
import tensorflow as tf
import os
import random
import numpy as np

def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

SEED = 22
seed_everything(seed=SEED)

In [2]:
from functools import reduce
from itertools import accumulate

landmark_lens = (
    (33, 4),
    (468, 3),
    (21, 3),
    (21, 3)
)
landmark_locs = list(accumulate(landmark_lens, lambda a, b: a + b[0]*b[1], initial=0))
landmarks_len = reduce(lambda r, loc: r + loc[0] * loc[1], landmark_lens, 0)
print(landmark_locs)

[0, 132, 1536, 1599, 1662]


In [3]:
data_folder = 'tracks_binary'
labels = [label for label in os.listdir(data_folder) if os.path.isdir(f'{data_folder}/{label}')]
NUM_CLASSES = len(labels)

labels_tensor = tf.constant(labels)
ids_tensor = tf.constant(range(len(labels)))

ids_from_labels = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        labels_tensor,
        ids_tensor
    ),
    default_value=-1
)

labels_from_ids = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        ids_tensor,
        labels_tensor
    ),
    default_value=""
)

def to_categorical(label):
    return tf.one_hot(
        ids_from_labels.lookup(label),
        depth=NUM_CLASSES
    )

In [4]:
def process_binary(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]

    raw = tf.io.read_file(file_path)
    data = tf.io.decode_raw(raw, tf.float32)
    data = tf.reshape(data, [-1, landmarks_len])

    pose = tf.reshape(data[:, 0:132], [-1, 33, 4])
    face = tf.reshape(data[:, 132:1536], [-1, 468, 3])
    lh = tf.reshape(data[:, 1536:1599], [-1, 21, 3])
    rh = tf.reshape(data[:, 1599:1662], [-1, 21, 3])

    return (pose, face, lh, rh), to_categorical(label)

In [5]:
from tensorflow import reduce_max, reduce_min

FRAMES = 64

def flatten(x):
    pose = tf.reshape(x[0], shape=[-1, 132])
    face = tf.reshape(x[1], shape=[-1, 1404])
    lh = tf.reshape(x[2], shape=[-1, 63])
    rh = tf.reshape(x[3], shape=[-1, 63])
    return tf.concat([pose, face, lh, rh], axis=1)


def random_window(x):
    def pad(x):
        missing = FRAMES - size
        start_pad = tf.math.ceil(missing / 2)
        end_pad = tf.math.floor(missing / 2)
        return tf.concat([
            tf.tile([x[0]], [start_pad, 1]),
            x,
            tf.tile([x[-1]], [end_pad, 1])
        ], axis=0)

    def random_slice(x):
        i = tf.random.uniform(shape=(), maxval=size+1-FRAMES, dtype=tf.int32)
        return x[i: i+FRAMES]

    size = tf.shape(x)[0]
    print(size)
    return tf.cond(
        size < FRAMES,
        lambda: pad(x),
        lambda: random_slice(x)
    )
    
def calc_bounding(pose, face, lh, rh):
    max_x = reduce_max(tf.stack([reduce_max(pose[:, :, :1]), reduce_max(face[:, :, :1]), reduce_max(lh[:, :, :1]), reduce_max(rh[:, :, :1])], axis=0))
    min_x = reduce_min(tf.stack([reduce_min(pose[:, :, :1]), reduce_min(face[:, :, :1]), reduce_min(lh[:, :, :1]), reduce_min(rh[:, :, :1])], axis=0))
    
    max_y = reduce_max(tf.stack([reduce_max(pose[:, :, 1:2]), reduce_max(face[:, :, 1:2]), reduce_max(lh[:, :, 1:2]), reduce_max(rh[:, :, 1:2])], axis=0))
    min_y = reduce_min(tf.stack([reduce_min(pose[:, :, 1:2]), reduce_min(face[:, :, 1:2]), reduce_min(lh[:, :, 1:2]), reduce_min(rh[:, :, 1:2])], axis=0))

    window = tf.cast((max_x - min_x, max_y - min_y), dtype=tf.float32)
    mid = ((max_x + min_x)/2, (max_y + min_y)/2)
    return (window, mid)

def scale(x, factor):
    pose, face, lh, rh = x[0], x[1], x[2], x[3]
    window, mid = calc_bounding(pose, face, lh, rh)
    scale = factor * window
    pose_shape, face_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(face), tf.shape(lh), tf.shape(rh)
    pose_center = tf.tile([[[mid[0], mid[1], 0, 0]]], [pose_shape[0], pose_shape[1], 1])
    face_center = tf.tile([[[mid[0], mid[1], 0]]], [face_shape[0], face_shape[1], 1])
    lh_center = tf.tile([[[mid[0], mid[1], 0]]], [lh_shape[0], lh_shape[1], 1])
    rh_center = tf.tile([[[mid[0], mid[1], 0]]], [rh_shape[0], rh_shape[1], 1])
    
    pose_scale = tf.tile([[[scale[0], scale[1], 1, 1]]], [pose_shape[0], pose_shape[1], 1])
    face_scale = tf.tile([[[scale[0], scale[1], 1]]], [face_shape[0], face_shape[1], 1])
    lh_scale = tf.tile([[[scale[0], scale[1], 1]]], [lh_shape[0], lh_shape[1], 1])
    rh_scale = tf.tile([[[scale[0], scale[1], 1]]], [rh_shape[0], rh_shape[1], 1])
    
    scaled_pose = pose_center + (pose - pose_center) * pose_scale
    scaled_face = face_center + (face - face_center) * face_scale
    scaled_lh = lh_center + (lh - lh_center) * lh_scale
    scaled_rh = rh_center + (rh - rh_center) * rh_scale

    return (scaled_pose, scaled_face, scaled_lh, scaled_rh)

def random_translation(x):
    pose, face, lh, rh = x[0], x[1], x[2], x[3]
    magnitude = tf.random.uniform(shape=[2], minval=-0.25, maxval=0.25)
    pose_shape, face_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(face), tf.shape(lh), tf.shape(rh)
    
    pose_trans = tf.tile([[[magnitude[0], magnitude[1], 0, 0]]], [pose_shape[0], pose_shape[1], 1])
    face_trans = tf.tile([[[magnitude[0], magnitude[1], 0]]], [face_shape[0], face_shape[1], 1])
    lh_trans = tf.tile([[[magnitude[0], magnitude[1], 0]]], [lh_shape[0], lh_shape[1], 1])
    rh_trans = tf.tile([[[magnitude[0], magnitude[1], 0]]], [rh_shape[0], rh_shape[1], 1])

    return (pose+pose_trans, face+face_trans, lh+lh_trans, rh+rh_trans)
    
def flip(x):
    pose, face, lh, rh = x[0], x[1], x[2], x[3]
    pose_shape, face_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(face), tf.shape(lh), tf.shape(rh)
    
    pose_neg = tf.tile([[[-1.0, 1, 1, 1]]], [pose_shape[0], pose_shape[1], 1])
    face_neg = tf.tile([[[-1.0, 1, 1]]], [face_shape[0], face_shape[1], 1])
    lh_neg = tf.tile([[[-1.0, 1, 1]]], [lh_shape[0], lh_shape[1], 1])
    rh_neg = tf.tile([[[-1.0, 1, 1]]], [rh_shape[0], rh_shape[1], 1])
    
    pose_trans = tf.tile([[[1.0, 0, 0, 0]]], [pose_shape[0], pose_shape[1], 1])
    face_trans = tf.tile([[[1.0, 0, 0]]], [face_shape[0], face_shape[1], 1])
    lh_trans = tf.tile([[[1.0, 0, 0]]], [lh_shape[0], lh_shape[1], 1])
    rh_trans = tf.tile([[[1.0, 0, 0]]], [rh_shape[0], rh_shape[1], 1])

    flipped_pose = pose_trans + pose * pose_neg
    flipped_face = face_trans + face * face_neg
    flipped_lh = lh_trans + lh * lh_neg
    flipped_rh = rh_trans + rh * rh_neg
    
    return (flipped_pose, flipped_face, flipped_lh, flipped_rh)
    
def prepare(ds, shuffle=False, augment=False):
    if augment:
        ds = ds.map(lambda x, y: (random_translation(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.map(lambda x, y: (scale(x, 0.1), y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.map(lambda x, y: (flip(x), y), num_parallel_calls=tf.data.AUTOTUNE)
               
    ds = ds.map(lambda x, y: (flatten(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.map(lambda x, y: (random_window(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000, seed=SEED)
        
    ds = ds.batch(32)

    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [6]:
def get_ds_split(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
  assert (train_split + test_split + val_split) == 1
  
  if shuffle:
    ds = ds.shuffle(shuffle_size, seed=SEED)
  
  train_size = int(train_split * ds_size)
  val_size = int(val_split * ds_size)
  
  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(val_size)
  test_ds = ds.skip(train_size).skip(val_size)
  
  return train_ds, val_ds, test_ds

In [7]:
import math

ds = tf.data.Dataset.list_files('tracks_binary_manual/*/*')
ds = ds.map(process_binary)
print(ds)
train_ds, val_ds, test_ds = get_ds_split(ds, len(ds))
print(train_ds)

train_ds = prepare(train_ds, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)


<MapDataset shapes: (((None, 33, 4), (None, 468, 3), (None, 21, 3), (None, 21, 3)), (10,)), types: ((tf.float32, tf.float32, tf.float32, tf.float32), tf.float32)>
<TakeDataset shapes: (((None, 33, 4), (None, 468, 3), (None, 21, 3), (None, 21, 3)), (10,)), types: ((tf.float32, tf.float32, tf.float32, tf.float32), tf.float32)>
Tensor("strided_slice:0", shape=(), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)


In [12]:
train_ds

<PrefetchDataset shapes: ((None, None, 1662), (None, 10)), types: (tf.float32, tf.float32)>

In [13]:
val_ds

<PrefetchDataset shapes: ((None, None, 1662), (None, 10)), types: (tf.float32, tf.float32)>

In [14]:
print(len(train_ds), len(val_ds), len(test_ds))

5 1 1


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from wandb.keras import WandbCallback
import wandb

In [16]:
wandb.init(
  project="bi-LSTM",
  entity="richardsonqiu", 
  config={
    "lstm_1": 120,
    "layer_1": 120,
    "act_1": "relu",
    
    "lstm_2": 130,
    # "dropout_2": 0.1,
    "layer_2": 130,
    "act_2": "relu",
        
    "lstm_3": 200,
    "dropout_3": 0.2,
    
    "last_layer": NUM_CLASSES,
    "last_act": "softmax",
    
    "optimizer": "adam",
    "init_lr": 0.01,
    "loss": "categorical_crossentropy",
    "metric": "accuracy",
    "epoch": 1000,
    "batch_size": 32,
    "data": "manual"  
    })
config = wandb.config

model = Sequential()
model.add(Bidirectional(LSTM(config.lstm_1, return_sequences=True), input_shape=(FRAMES, 1662)))
model.add(Dense(config.layer_1, activation=config.act_1))
model.add(Bidirectional(LSTM(config.lstm_2, return_sequences=True)))
model.add(Dense(config.layer_2, activation=config.act_2))
model.add(Bidirectional(LSTM(config.lstm_3, return_sequences=False, dropout=config.dropout_3)))
model.add(Dense(config.last_layer, activation=config.last_act))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: richardsonqiu (use `wandb login --relogin` to force relogin)


In [17]:
opt = keras.optimizers.Adam(learning_rate=config.init_lr)
model.compile(optimizer=opt, loss=config.loss, metrics=[config.metric])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 64, 240)          1711680   
 l)                                                              
                                                                 
 dense (Dense)               (None, 64, 120)           28920     
                                                                 
 bidirectional_1 (Bidirectio  (None, 64, 260)          261040    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 64, 130)           33930     
                                                                 
 bidirectional_2 (Bidirectio  (None, 400)              529600    
 nal)                                                            
                                                        

In [19]:
# TRIAL = 2
# log_dir = os.path.join('Heh/{}'.format(TRIAL))
# tb_callback = TensorBoard(log_dir=log_dir)
es_callback = EarlyStopping(monitor='val_loss', patience=20)
lr_callback = ReduceLROnPlateau(monitor='val_loss', patience=20, factor=0.5, min_lr=1e-6)
wandb_callback = WandbCallback(log_evaluation=True)

In [20]:
history = model.fit(train_ds, validation_data=val_ds, epochs=config.epoch, callbacks=[lr_callback, wandb_callback])

wandb: WARNING WandbCallback is unable to log validation data. When using a generator for validation_data, you must pass validation_steps


Epoch 1/1000
5/5 [==============================] - 22s 1s/step - loss: 3.2901 - accuracy: 0.0688 - val_loss: 2.8605 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 2/1000
5/5 [==============================] - 2s 275ms/step - loss: 2.5017 - accuracy: 0.0812 - val_loss: 2.3611 - val_accuracy: 0.1000 - lr: 0.0100
Epoch 3/1000
5/5 [==============================] - 2s 311ms/step - loss: 2.3904 - accuracy: 0.0688 - val_loss: 2.4068 - val_accuracy: 0.0500 - lr: 0.0100
Epoch 4/1000
5/5 [==============================] - 2s 288ms/step - loss: 2.3322 - accuracy: 0.0750 - val_loss: 2.3524 - val_accuracy: 0.1000 - lr: 0.0100
Epoch 5/1000
5/5 [==============================] - 1s 240ms/step - loss: 2.3198 - accuracy: 0.1000 - val_loss: 2.3009 - val_accuracy: 0.1500 - lr: 0.0100
Epoch 6/1000
5/5 [==============================] - 1s 247ms/step - loss: 2.3098 - accuracy: 0.1063 - val_loss: 2.3360 - val_accuracy: 0.1500 - lr: 0.0100
Epoch 7/1000
5/5 [==============================] - 2s 261ms/step - 

KeyboardInterrupt: 

In [15]:
model.save('steven{}.h5'.format(TRIAL))

In [16]:
print(len(train_ds), len(val_ds))

8 2
